In [ ]:
!pip install rdkit
!pip install py3Dmol
!pip install pubchempy
!pip install dgllife
!pip install dgl

In [ ]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import Draw # For 2D representation of molecule
import py3Dmol # For 3D representation of molecule
import pubchempy as pcp
# fetching the molecule from PubChem database 
compound = pcp.get_compounds ('Benzene', 'name')[0]

# fetching the SMILES notation of the molecule
smiles_compound = compound.canonical_smiles
print('Canonical structure:', smiles_compound)

# Generating the Molecular connectivities from SMILES
mol = Chem.MolFromSmiles (smiles_compound)
# Adding H-atoms to fulfil valency of C -atom 
mol =  Chem.AddHs (mol)

# Visualize molecule in 3D
viewer = py3Dmol.view()
viewer.addModel (Chem.MolToMolBlock (mol), 'mol')
viewer.setStyle({'stick': {}})
viewer.zoomTo()
viewer.show()

# Visualize molecule in 2D
mol_image = Draw.MolToImage (mol) 
mol_image.show()

# Graph Neural Network

Loading a pre-trained Graph Neural Network

In [ ]:
import torch
import dgllife
from dgllife.data import Tox21
from dgllife.model import load_pretrained
from dgllife.utils import smiles_to_bigraph, CanonicalAtomFeaturizer

dataset = Tox21(smiles_to_bigraph, CanonicalAtomFeaturizer())

# Loading the Pretrained model 
model = load_pretrained('GCN_Tox21') 
model.eval()

smiles, g, label, mask = dataset[0]
feats = g.ndata.pop('h')
label_pred = model(g, feats)
print(smiles)                   # CCOc1ccc2nc(S(N)(=O)=O)sc2c1
print(label_pred[:, mask != 0]) 